In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/finetune-whisper')

Mounted at /content/drive/


In [ ]:
!pip install datasets>=2.6.1
!pip install evaluate>=0.30
!pip install jiwer
!pip install striprtf
!pip install tiktoken
!pip install git+https://github.com/openai/whisper.git
!pip install Unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-wfd4blo9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-wfd4blo9
  Resolved https://github.com/openai/whisper.git to commit b91c907694f96a3fb9da03d4bbdc83fbcd3a40a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798285 sha256=0f8f31ef42142c711c3b426281cf9f428302002b055a48e945bdc7b9cb2d3a0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-zjqqo0i3/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whispe

In [ ]:
import locale
import os
import re
from striprtf.striprtf import rtf_to_text
from datasets import Dataset

locale.getpreferredencoding = lambda: "UTF-8"

def create_dataset(dir, task):

    diretorio = dir
    padrao_dialogo = re.compile(r'^(\w+):\s(.*)$')

    arquivos_dialogo = {}
    metadata = {
        "audio": [],
        "text": []
    }


    for arquivo in os.listdir(diretorio):

        if arquivo.endswith(".RTF"):
            conversa = ""
            caminho = os.path.join(diretorio, arquivo)

            with open(caminho) as infile:
                rtf_data = infile.read()
                texto = rtf_to_text(rtf_data)

            linhas = texto.split('\n')
            dialogos = [padrao_dialogo.match(linha) for linha in linhas]
            dialogos = [f"{match.group(2)} " for match in dialogos if match]
            conversa += "".join(dialogos)

            arquivo = arquivo.replace("RTF", "WAV")
            arquivos_dialogo["./wav/"+ task + "/" + arquivo] = conversa


    for key, value in arquivos_dialogo.items():

        if value != "":

            metadata["audio"].append(key)
            metadata["text"].append(value)


    audio_dataset = Dataset.from_dict(metadata)

    return audio_dataset




In [ ]:
train_dataset = create_dataset("./wav/train", "train")
test_dataset = create_dataset("./wav/test", "test")

In [ ]:
from datasets import DatasetDict

sis_dataset = DatasetDict()

sis_dataset["train"] = create_dataset("./wav/train", "train")
sis_dataset["test"] = create_dataset("./wav/test", "test")

print(sis_dataset)

In [ ]:
from datasets import Audio

sis_dataset_sr16 = sis_dataset.cast_column("audio", Audio(sampling_rate=16000))
references = sis_dataset_sr16["test"]['text']

In [ ]:
import whisper

model = whisper.load_model("large")

predictions = []
for audio in sis_dataset_sr16["test"]['audio']:
  result = model.transcribe(audio['path'], language='pt')
  predictions.append(result['text'])

In [ ]:
import re
import unicodedata

def remover_caracteres_especiais(texto):
    texto = re.sub('[^\w\s]', '', texto)
    return texto

def remover_acentos(texto):
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

def converter_para_minusculas(texto):
    texto = texto.lower()
    return texto

def remover_espacos_extras(texto):
    texto = re.sub('\s+', ' ', texto)
    return texto

def remover_digitos(texto):
    texto = re.sub(r'\d+', '', texto)  # Remove dígitos
    return texto

def remover_espacos_inicio_fim(texto):
    texto = texto.strip()  # Remove espaços em branco no início e no fim
    return texto

def normalizar_transcricoes(transcricoes):
    transcricoes_normalizadas = []
    for transcricao in transcricoes:
        texto_normalizado = remover_caracteres_especiais(transcricao)
        texto_normalizado = remover_acentos(texto_normalizado)
        texto_normalizado = converter_para_minusculas(texto_normalizado)
        texto_normalizado = remover_espacos_extras(texto_normalizado)
        texto_normalizado = remover_digitos(texto_normalizado)
        texto_normalizado = remover_espacos_inicio_fim(texto_normalizado)
        transcricoes_normalizadas.append(texto_normalizado)
    return transcricoes_normalizadas


In [ ]:
from evaluate import load
wer = load("wer")
wer_score = wer.compute(predictions=normalizar_transcricoes(predictions), references=normalizar_transcricoes(references))
#print(wer_score)
print(f"WER: {wer_score * 100:.2f} %")

WER: 44.05 %


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00


In [ ]:

from transformers import pipeline

transcriber = pipeline(
  "automatic-speech-recognition",
  model="jonatasgrosman/whisper-large-pt-cv11"
)

transcriber.model.config.forced_decoder_ids = (
  transcriber.tokenizer.get_decoder_prompt_ids(
    language="pt",
    task="transcribe"
  )
)

#transcription = transcriber("path/to/my_audio.wav")


In [ ]:
import whisper

#model = whisper.load_model("large")

predictions = []
for audio in sis_dataset_sr16["test"]['audio']:
  result = transcriber(audio['path'])
  predictions.append(result['text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to cont

In [ ]:
from evaluate import load
wer = load("wer")
wer_score = wer.compute(predictions=normalizar_transcricoes(predictions), references=normalizar_transcricoes(references))
#print(wer_score)
print(f"WER: {wer_score * 100:.2f} %")

WER: 88.99 %


In [ ]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "pt"
MODEL_ID = "jonatasgrosman/wav2vec2-xls-r-1b-portuguese"
SAMPLES = 10

test_dataset = sis_dataset_sr16['test']

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch['audio']["path"], sr=16_000)
    batch["speech"] = speech_array
    batch["text"] = batch["text"].upper()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

predictions = predicted_sentences

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
from evaluate import load
wer = load("wer")
wer_score = wer.compute(predictions=normalizar_transcricoes(predictions), references=normalizar_transcricoes(references))
print(f"WER: {wer_score * 100:.2f} %")

WER: 60.35 %
